In [9]:
import time
import tensorflow as tf

LWE_N=1300
LWE_M=2048
LWE_L=512
Q=2048
P=512
_16_LOG_Q=5
_32_LOG_Q=21
_16_LOG_T= 15
KeyTime=0

In [10]:
def gen_pk_A():     
    global KeyTime
    Key_A_start=time.time()
    pk_A=tf.random_uniform([LWE_M, LWE_N], minval=0, maxval=Q, dtype=tf.int32, seed=None, name=None)
    pk_Ai32=tf.bitwise.left_shift(pk_A, _32_LOG_Q)
    pk_A=tf.cast(pk_A, tf.uint16)    
    KeyTime=time.time()-Key_A_start
    return pk_A, pk_Ai32

In [11]:
def gen_sk_S():
    global KeyTime
    Key_S_start=time.time()
    x=tf.random_uniform([LWE_N, LWE_L], minval=0, maxval=1001, dtype=tf.int32)
    c1=tf.fill([LWE_N, LWE_L], -1)
    r1=tf.where(tf.less(x, 125), c1, c1+1)
    r2=tf.where(tf.greater(x, 875), c1+2, r1)
    sk_S=tf.cast(r2, tf.uint16)
    sk_Si32=tf.cast(sk_S, tf.int32)
    KeyTime=time.time()-Key_S_start
    return sk_S, sk_Si32

In [14]:
def gen_E():
    rnd=tf.random_uniform([LWE_M, LWE_L], minval=0, maxval=1024, dtype=tf.int32, seed=None, name=None)
    sign=tf.random_uniform([LWE_M, LWE_L], minval=0, maxval=2, dtype=tf.int32, seed=None, name=None)
    sign=2*sign-1
    cdf1=tf.fill([LWE_M, LWE_L], 3)

    r1=tf.where(tf.less(rnd, 1009), cdf1-1, cdf1)
    r2=tf.where(tf.less(rnd, 875), cdf1-2, r1)
    r3=tf.where(tf.less(rnd, 381), cdf1-3, r2)

    r9=r3*sign
    E_matrix=tf.cast(r9, tf.uint16)
    E_matrix=tf.bitwise.left_shift(E_matrix, _16_LOG_Q) 
    
    E_matrix=tf.cast(E_matrix, tf.int32)

    return E_matrix

In [15]:
def gen_pk_B(pk_Ai32, sk_Si32):
    global KeyTime
    Key_B_start=time.time()
    E_matrix=gen_E()
    pk_B=tf.subtract(E_matrix, tf.matmul(pk_Ai32, sk_Si32, b_is_sparse=True))
    pk_B=tf.cast(pk_B, tf.uint16)
    pk_Bi32=tf.cast(pk_B, tf.int32)
    KeyTime=time.time()-Key_B_start
    return pk_B, pk_Bi32 

In [17]:
iter=5000
totalkey=0
for i in range(iter):
    #######Key_generator##############
    pk_A, pk_Ai32=gen_pk_A()
    sk_S, sk_Si32=gen_sk_S()
    pk_B, pk_Bi32=gen_pk_B(pk_Ai32, sk_Si32)
    totalkey=totalkey+KeyTime
    
print("Keygen : ", round(totalkey*1000/iter,3), "ms")


Keygen :  28.956 ms
